In [1]:
from datasets import load_dataset
import pandas as pd
import pickle
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from rdkit import Chem

train = load_dataset('lewtun/drug-reviews', split='train')
test = load_dataset('lewtun/drug-reviews', split='test')
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)

In [6]:
df_pcc = pd.read_csv('../../Datasets/drugs/pccs-pubchem/test.tsv', sep='\t')
df_pcc_tr = pd.read_csv('../../Datasets/drugs/pccs-pubchem/train.tsv', sep='\t')
df_pcc.head()
df_pcc_tr.head()
df_pcc['Name'].tolist()[4]

,CID,Name,SMILES,InChI,Name type
0,1107,"2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17-hexade...",OC1CCC2C(CCC3C4CCCC4CCC23)C1,InChI=1S/C17H28O/c18-13-6-9-15-12(10-13)5-8-16...,IUPAC
1,1107,SureCN4544647,OC1CCC2C(CCC3C4CCCC4CCC23)C1,InChI=1S/C17H28O/c18-13-6-9-15-12(10-13)5-8-16...,synonym
2,2530,ACM3147146|1,Cc1ccc(O)c(N=Nc2c(O)cc(S(=O)(=O)O)c3ccccc23)c1,InChI=1S/C17H14N2O5S/c1-10-6-7-14(20)13(8-10)1...,synonym
3,2530,(4Z)-4-[(2-hydroxy-5-methylphenyl)hydrazinylid...,Cc1ccc(O)c(N=Nc2c(O)cc(S(=O)(=O)O)c3ccccc23)c1,InChI=1S/C17H14N2O5S/c1-10-6-7-14(20)13(8-10)1...,synonym
4,4443,N-[[TRANS-4-(1-METHYLETHYL)CYCLOHEXYL]CARBONYL...,CC(C)C1CCC(C(=O)NC(Cc2ccccc2)C(=O)O)CC1,InChI=1S/C19H27NO3/c1-13(2)15-8-10-16(11-9-15)...,synonym


,CID,Name,SMILES,InChI
0,101825401,"2-[Bis(3,5-dibromo-4-hydroxy-2-methylphenyl)hy...",Cc1c(C(O)(c2ccccc2S(=O)(=O)O)c2cc(Br)c(O)c(Br)...,InChI=1S/C21H16Br4O6S.Na/c1-9-12(7-14(22)19(26...
1,101825403,"1,1',1''-(1,3,5,5-Tetramethyl-1-pentene-1,3,5-...",C/C(=C/C(C)(CC(C)(C)c1ccccc1)c1ccccc1)c1ccccc1,InChI=1S/C27H30/c1-22(23-14-8-5-9-15-23)20-27(...
2,101825404,"3-Potassiopentane-2,4-dione",CC(=O)C=C(C)[O].[K],"InChI=1S/C5H7O2.K/c1-4(6)3-5(2)7;/h3H,1-2H3;"
3,101825405,Benzenesulfonic acid ammonium salt,[NH3+]O[S+]([O])(=O)c1ccccc1,"InChI=1S/C6H8NO3S/c7-10-11(8,9)6-4-2-1-3-5-6/h..."
4,101825408,3-Methyl-4-phenyl-3-butenoic acid sodium salt,C/C(=C\c1ccccc1)CC(=O)[O-].[Na+],InChI=1S/C11H12O2.Na/c1-9(8-11(12)13)7-10-5-3-...


'N-[[TRANS-4-(1-METHYLETHYL)CYCLOHEXYL]CARBONYL]-D-'

0

In [17]:
def count_carbon_atoms(smiles, atom_name="C"):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:  # Invalid SMILES string
        return None
    carbon_atoms = [atom for atom in mol.GetAtoms() if atom.GetSymbol() == atom_name]
    return len(carbon_atoms)

n = 102
df_pcc['SMILES'].tolist()[n]
count_carbon_atoms(df_pcc['SMILES'].tolist()[n])

'COC(=O)C1=CCCN(N=O)C1'

7

In [15]:
df = df_pcc[~df_pcc['Name'].isin(df_pcc_tr['Name'].tolist())]
len(df)
len(df_pcc_tr), len(df_pcc)

195633

(5000000, 195660)

In [9]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_test = df_test.drop(['Unnamed: 0'], axis=1)
df_train['is_test'] = False
df_test['is_test'] = True

In [15]:
top_30_classes = df_test['condition'].value_counts().nlargest(30).index.tolist()
df_train_filtered = df_train[df_train['condition'].isin(top_30_classes)]
df_test_filtered = df_test[df_test['condition'].isin(top_30_classes)]
df_train_filtered['cat'] = df_train_filtered['condition'].astype('category').cat.codes
df_test_filtered['cat'] = df_test_filtered['condition'].astype('category').cat.codes
mapping_dict = dict(enumerate(df_test_filtered['condition'].astype('category').cat.categories))
mapping_dict_ = dict(enumerate(df_train_filtered['condition'].astype('category').cat.categories))

print(mapping_dict == mapping_dict_)
len(df_train_filtered), len(df_test_filtered)
df_train_filtered.head()

True


/Users/tianyuhan/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/tianyuhan/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


(107943, 36124)

,drugName,condition,review,rating,date,usefulCount,is_test,cat
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,False,0
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,False,6
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,False,6
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,False,21
6,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1.0,"March 7, 2017",5,False,12


In [16]:
df = pd.concat([df_train_filtered, df_test_filtered])
len(df)

144067

In [17]:
df.to_csv('./data/entity_datasets/drug.csv', index=False)

In [18]:
top_30_classes

['Birth Control',
 'Depression',
 'Pain',
 'Anxiety',
 'Acne',
 'Bipolar Disorde',
 'Weight Loss',
 'Insomnia',
 'Obesity',
 'ADHD',
 'Emergency Contraception',
 'Vaginal Yeast Infection',
 'Diabetes, Type 2',
 'High Blood Pressure',
 'Smoking Cessation',
 'Abnormal Uterine Bleeding',
 'Bowel Preparation',
 'Migraine',
 'ibromyalgia',
 'Anxiety and Stress',
 'Constipation',
 'Major Depressive Disorde',
 'Chronic Pain',
 'Panic Disorde',
 'Migraine Prevention',
 'Urinary Tract Infection',
 'Opiate Dependence',
 'Osteoarthritis',
 'Muscle Spasm',
 'Erectile Dysfunction']

In [9]:
path = '/home/than/DeepLearning/world-models/results/tuned/Llama-2-70b-hf/drug/condition/probes/lsq_train.none.none.last.condition.p'
probe_results = pickle.load(open(path, 'rb'))
df = probe_results['projections']
df[70].head()

,projection,is_test
0,0,False
1,6,False
2,6,False
3,21,False
4,6,False
